In [1]:
import tensorflow as tf

In [2]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import imblearn
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('econet dataset/train.csv')
unique_locations = train_df['Station'].unique()

In [4]:
train_df['Ob'] = pd.to_datetime(train_df['Ob'])

In [8]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
label_encoder = LabelEncoder()
features = ['Station', 'Ob', 'value', 'measure', 'R_flag', 'I_flag', 'Z_flag', 'B_flag']
temp_df = train_df.loc[:]
label_encoder = LabelEncoder()
temp_df['Station'] = label_encoder.fit_transform(train_df['Station'])
temp_df['measure'] = label_encoder.fit_transform(train_df['measure'])
temp_df['Ob'] = train_df['Ob'].values.astype(float)

x = temp_df.loc[:, features].values
y = temp_df.loc[:,['target']].values
x = StandardScaler().fit_transform(x)

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
principalComponents = pca.fit_transform(x)
p_df = pd.DataFrame(principalComponents)

In [10]:
finalDf = pd.concat([p_df, temp_df[['target']]], axis = 1)
print(finalDf.head())

          0         1         2         3         4         5         6  \
0 -0.326512  0.946291  0.364007  2.680728 -0.883147  0.672595 -0.590456   
1 -0.325359  0.945896  0.362786  2.679443 -0.883103  0.673863 -0.591416   
2 -0.325825  0.946202  0.362537  2.679315 -0.883461  0.673131 -0.590933   
3 -0.327426  0.947174  0.362091  2.679230 -0.884507  0.670732 -0.589325   
4 -0.289066  0.986795  0.054464  2.403466 -1.005693  0.633474 -0.586918   

   target  
0   False  
1   False  
2   False  
3   False  
4   False  


In [11]:
from scipy import stats

def create_windows(X, y, time_steps=1, step=1):
    Xs, ys = [], []
      
    for i in range(0, len(X) - time_steps + 1, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])

    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [30]:
# y = label_encoder.fit_transform(finalDf['target'])
y = finalDf['target']
X = finalDf[[0, 1, 2, 3, 4, 5, 6]]
print(type(y), type(X))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [31]:
print(X.shape, y.shape)

(6593274, 7) (6593274,)


In [32]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

over = SMOTE(sampling_strategy='not majority')
under = RandomUnderSampler(sampling_strategy='majority')

X, y = under.fit_resample(X, y)
print(X.shape, y.shape)
X, y = over.fit_resample(X, y)
print(X.shape, y.shape)

(470344, 7) (470344,)
(470344, 7) (470344,)


In [33]:
pd.DataFrame(y).value_counts()

target
False     235172
True      235172
dtype: int64

In [34]:
TIME_STEPS = 40
STEP = 1

X, y = create_windows(X, y, time_steps=TIME_STEPS,step=STEP)

In [35]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Converting label to OneHot Encoding
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_test = enc.transform(y_test)